Alternating between clustering of the image descriptors and updating the weights of the convnet by predicting the cluster assignment.

K-means(need to read implement again)

In [ ]:
import torch
import torch.nn.functional as F


def deepclusterv2_loss_func(
    outputs: torch.Tensor, assignments: torch.Tensor, temperature: float = 0.1
) -> torch.Tensor:
    """Computes DeepClusterV2's loss given a tensor containing logits from multiple views
    and a tensor containing cluster assignments from the same multiple views.
    Args:
        outputs (torch.Tensor): tensor of size PxVxNxC where P is the number of prototype
            layers and V is the number of views.
        assignments (torch.Tensor): tensor of size PxVxNxC containing the assignments
            generated using k-means.
        temperature (float, optional): softmax temperature for the loss. Defaults to 0.1.
    Returns:
        torch.Tensor: DeepClusterV2 loss.
    """
    loss = 0
    for h in range(outputs.size(0)):
        scores = outputs[h].view(-1, outputs.size(-1)) / temperature
        targets = assignments[h].repeat(outputs.size(1)).to(outputs.device, non_blocking=True)
        loss += F.cross_entropy(scores, targets, ignore_index=-1)
    return loss / outputs.size(0)

In [ ]:
from typing import Any, Sequence

import numpy as np
import torch
import torch.distributed as dist
import torch.nn.functional as F
from scipy.sparse import csr_matrix


class KMeans:
    def __init__(
        self,
        world_size: int,
        rank: int,
        num_crops: int,
        dataset_size: int,
        proj_features_dim: int,
        num_prototypes: int,
        kmeans_iters: int = 10,
    ):
        """Class that performs K-Means on the hypersphere.

        Args:
            world_size (int): world size.
            rank (int): rank of the current process.
            num_crops (int): number of crops.
            dataset_size (int): total size of the dataset (number of samples).
            proj_features_dim (int): number of dimensions of the projected features.
            num_prototypes (int): number of prototypes.
            kmeans_iters (int, optional): number of iterations for the k-means clustering.
                Defaults to 10.
        """
        self.world_size = world_size
        self.rank = rank
        self.num_crops = num_crops
        self.dataset_size = dataset_size
        self.proj_features_dim = proj_features_dim
        self.num_prototypes = num_prototypes
        self.kmeans_iters = kmeans_iters

    @staticmethod
    def get_indices_sparse(data: np.ndarray):
        cols = np.arange(data.size)
        M = csr_matrix((cols, (data.ravel(), cols)), shape=(int(data.max()) + 1, data.size))
        return [np.unravel_index(row.data, data.shape) for row in M]

    def cluster_memory(
        self,
        local_memory_index: torch.Tensor,
        local_memory_embeddings: torch.Tensor,
    ) -> Sequence[Any]:
        """Performs K-Means clustering on the hypersphere and returns centroids and
        assignments for each sample.

        Args:
            local_memory_index (torch.Tensor): memory bank cointaining indices of the
                samples.
            local_memory_embeddings (torch.Tensor): memory bank cointaining embeddings
                of the samples.

        Returns:
            Sequence[Any]: assignments and centroids.
        """
        j = 0
        device = local_memory_embeddings.device
        assignments = -torch.ones(len(self.num_prototypes), self.dataset_size).long()
        centroids_list = []
        with torch.no_grad():
            for i_K, K in enumerate(self.num_prototypes):
                # run distributed k-means

                # init centroids with elements from memory bank of rank 0
                centroids = torch.empty(K, self.proj_features_dim).to(device, non_blocking=True)
                if self.rank == 0:
                    random_idx = torch.randperm(len(local_memory_embeddings[j]))[:K]
                    assert len(random_idx) >= K, "please reduce the number of centroids"
                    centroids = local_memory_embeddings[j][random_idx]
                if dist.is_available() and dist.is_initialized():
                    dist.broadcast(centroids, 0)

                for n_iter in range(self.kmeans_iters + 1):

                    # E step
                    dot_products = torch.mm(local_memory_embeddings[j], centroids.t())
                    _, local_assignments = dot_products.max(dim=1)

                    # finish
                    if n_iter == self.kmeans_iters:
                        break

                    # M step
                    where_helper = self.get_indices_sparse(local_assignments.cpu().numpy())
                    counts = torch.zeros(K).to(device, non_blocking=True).int()
                    emb_sums = torch.zeros(K, self.proj_features_dim).to(device, non_blocking=True)
                    for k in range(len(where_helper)):
                        if len(where_helper[k][0]) > 0:
                            emb_sums[k] = torch.sum(
                                local_memory_embeddings[j][where_helper[k][0]],
                                dim=0,
                            )
                            counts[k] = len(where_helper[k][0])
                    if dist.is_available() and dist.is_initialized():
                        dist.all_reduce(counts)
                        dist.all_reduce(emb_sums)
                    mask = counts > 0
                    centroids[mask] = emb_sums[mask] / counts[mask].unsqueeze(1)

                    # normalize centroids
                    centroids = F.normalize(centroids, dim=1, p=2)

                centroids_list.append(centroids)

                if dist.is_available() and dist.is_initialized():
                    # gather the assignments
                    assignments_all = torch.empty(
                        self.world_size,
                        local_assignments.size(0),
                        dtype=local_assignments.dtype,
                        device=local_assignments.device,
                    )
                    assignments_all = list(assignments_all.unbind(0))

                    dist_process = dist.all_gather(
                        assignments_all, local_assignments, async_op=True
                    )
                    dist_process.wait()
                    assignments_all = torch.cat(assignments_all).cpu()

                    # gather the indexes
                    indexes_all = torch.empty(
                        self.world_size,
                        local_memory_index.size(0),
                        dtype=local_memory_index.dtype,
                        device=local_memory_index.device,
                    )
                    indexes_all = list(indexes_all.unbind(0))
                    dist_process = dist.all_gather(indexes_all, local_memory_index, async_op=True)
                    dist_process.wait()
                    indexes_all = torch.cat(indexes_all).cpu()

                else:
                    assignments_all = local_assignments
                    indexes_all = local_memory_index

                # log assignments
                assignments[i_K][indexes_all] = assignments_all

                # next memory bank to use
                j = (j + 1) % self.num_crops

        return assignments, centroids_list


In [ ]:
import argparse
from typing import Any, Dict, List, Sequence

import torch
import torch.nn as nn
import torch.nn.functional as F
from solo.losses.deepclusterv2 import deepclusterv2_loss_func
from solo.methods.base import BaseModel
from solo.utils.kmeans import KMeans


class DeepClusterV2(BaseModel):
    def __init__(
        self,
        output_dim: int,
        proj_hidden_dim: int,
        num_prototypes: Sequence[int],
        temperature: float,
        kmeans_iters: int,
        **kwargs,
    ):
        """Implements DeepCluster V2 (https://arxiv.org/abs/2006.09882).
        Args:
            output_dim (int): number of dimensions of the projected features.
            proj_hidden_dim (int): number of neurons in the hidden layers of the projector.
            num_prototypes (Sequence[int]): number of prototypes.
            temperature (float): temperature for the softmax.
            kmeans_iters (int): number of iterations for k-means clustering.
        """

        super().__init__(**kwargs)

        self.output_dim = output_dim
        self.temperature = temperature
        self.num_prototypes = num_prototypes
        self.kmeans_iters = kmeans_iters

        # projector
        self.projector = nn.Sequential(
            nn.Linear(self.features_dim, proj_hidden_dim),
            nn.BatchNorm1d(proj_hidden_dim),
            nn.ReLU(),
            nn.Linear(proj_hidden_dim, output_dim),
        )

        # prototypes
        self.prototypes = nn.ModuleList(
            [nn.Linear(output_dim, np, bias=False) for np in num_prototypes]
        )
        # normalize and set requires grad to false
        for proto in self.prototypes:
            for params in proto.parameters():
                params.requires_grad = False
            proto.weight.copy_(F.normalize(proto.weight.data.clone(), dim=-1))

    @staticmethod
    def add_model_specific_args(parent_parser: argparse.ArgumentParser) -> argparse.ArgumentParser:
        parent_parser = super(DeepClusterV2, DeepClusterV2).add_model_specific_args(parent_parser)
        parser = parent_parser.add_argument_group("deepclusterv2")

        # projector
        parser.add_argument("--output_dim", type=int, default=128)
        parser.add_argument("--proj_hidden_dim", type=int, default=2048)

        # parameters
        parser.add_argument("--temperature", type=float, default=0.1)
        parser.add_argument("--num_prototypes", type=int, nargs="+", default=[3000, 3000, 3000])
        parser.add_argument("--kmeans_iters", type=int, default=10)

        return parent_parser

    @property
    def learnable_params(self) -> List[dict]:
        """Adds projector and prototypes parameters to the parent's learnable parameters.
        Returns:
            List[dict]: list of learnable parameters.
        """

        extra_learnable_params = [{"params": self.projector.parameters()}]
        return super().learnable_params + extra_learnable_params

    def on_train_start(self):
        """Gets the world size and initializes the memory banks."""
        #  k-means needs the world size and the dataset size
        self.world_size = self.trainer.world_size if self.trainer else 1
        self.dataset_size = getattr(self, "dali_epoch_size", None) or len(
            self.trainer.train_dataloader.dataset
        )

        # build k-means helper object
        self.kmeans = KMeans(
            world_size=self.world_size,
            rank=self.global_rank,
            num_crops=self.num_crops,
            dataset_size=self.dataset_size,
            proj_features_dim=self.output_dim,
            num_prototypes=self.num_prototypes,
            kmeans_iters=self.kmeans_iters,
        )

        # initialize memory banks
        size_memory_per_process = len(self.trainer.train_dataloader) * self.batch_size
        self.register_buffer(
            "local_memory_index",
            torch.zeros(size_memory_per_process).long().to(self.device, non_blocking=True),
        )
        self.register_buffer(
            "local_memory_embeddings",
            F.normalize(
                torch.randn(self.num_crops, size_memory_per_process, self.output_dim), dim=-1
            ).to(self.device, non_blocking=True),
        )

    def on_train_epoch_start(self) -> None:
        """Prepares assigments and prototype centroids for the next epoch."""

        if self.current_epoch == 0:
            self.assignments = -torch.ones(
                len(self.num_prototypes), self.dataset_size, device=self.device
            ).long()
        else:
            self.assignments, centroids = self.kmeans.cluster_memory(
                self.local_memory_index, self.local_memory_embeddings
            )
            for proto, centro in zip(self.prototypes, centroids):
                proto.weight.copy_(centro)

    def update_memory_banks(self, idxs: torch.Tensor, z: torch.Tensor, batch_idx: int) -> None:
        """Updates DeepClusterV2's memory banks of indices and features.
        Args:
            idxs (torch.Tensor): set of indices of the samples of the current batch.
            z (torch.Tensor): projected features of the samples of the current batch.
            batch_idx (int): batch index relative to the current epoch.
        """

        start_idx, end_idx = batch_idx * self.batch_size, (batch_idx + 1) * self.batch_size
        self.local_memory_index[start_idx:end_idx] = idxs
        for c, z_c in enumerate(z):
            self.local_memory_embeddings[c][start_idx:end_idx] = z_c.detach()

    def forward(self, X: torch.Tensor, *args, **kwargs) -> Dict[str, Any]:
        """Performs the forward pass of the encoder, the projector and the prototypes.
        Args:
            X (torch.Tensor): a batch of images in the tensor format.
        Returns:
            Dict[str, Any]:
                a dict containing the outputs of the parent,
                the projected features and the logits.
        """

        out = super().forward(X, *args, **kwargs)
        z = F.normalize(self.projector(out["feats"]))
        p = torch.stack([p(z) for p in self.prototypes])
        return {**out, "z": z, "p": p}

    def training_step(self, batch: Sequence[Any], batch_idx: int) -> torch.Tensor:
        """Training step for DeepClusterV2 reusing BaseModel training step.
        Args:
            batch (Sequence[Any]): a batch of data in the format of [img_indexes, [X], Y], where
                [X] is a list of size self.num_crops containing batches of images.
            batch_idx (int): index of the batch.
        Returns:
            torch.Tensor: total loss composed of DeepClusterV2 loss and classification loss.
        """

        idxs = batch[0]

        out = super().training_step(batch, batch_idx)
        class_loss = out["loss"]
        feats1, feats2 = out["feats"]

        z1 = F.normalize(self.projector(feats1))
        z2 = F.normalize(self.projector(feats2))

        p1 = torch.stack([proto(z1) for proto in self.prototypes])
        p2 = torch.stack([proto(z2) for proto in self.prototypes])

        # ------- deepclusterv2 loss -------
        preds = torch.stack([p1.unsqueeze(1), p2.unsqueeze(1)], dim=1)
        assignments = self.assignments[:, idxs]
        deepcluster_loss = deepclusterv2_loss_func(preds, assignments, self.temperature)

        # ------- update memory banks -------
        self.update_memory_banks(idxs, [z1, z2], batch_idx)

        self.log("train_deepcluster_loss", deepcluster_loss, on_epoch=True, sync_dist=True)

        return deepcluster_loss + class_loss